Test a potential measure of how "ligned up" the nodes are by:
* Writing a simple hierarchical clustering algorithm
* Graphing the way the variance between the clusters changes at each step
* Graphing the way the sum of the variance within the clusters changes at each step

Requirements:
* Network being measured must be open in Cytoscape

In [3]:
from py2cytoscape import cyrest
from statistics import *
from math import *
from numpy.random import randint

In [19]:
#Get data from current network
cy = cyrest.cyclient()
data = {}

#Rounding to hundredth of shortest node
#raw_data = cy.node.get_properties(network='CURRENT', nodeList='all', propertyList='Y Location, Height')
#min_height = min([prop['value'] for prop in entry['visualProperties'] if prop['visualProperty'] == 'NODE_HEIGHT'])
#magnitude = floor(log10(min_height / 100))

#if magnitude <= 0:
 #   for entry in raw_data:
  #      data[str(entry['SUID'])] = round(
   #         [
    #            prop['value'] 
     #           for prop in entry['visualProperties'] 
      #          if prop['visualProperty'] == 'NODE_Y_LOCATION'
       #     ][0],
        #    -magnitude
        #)
#else:
 #   for entry in raw_data:
  #      data[str(entry['SUID'])] = round(
   #         [
    #            prop['value'] 
     #           for prop in entry['visualProperties'] 
      #          if prop['visualProperty'] == 'NODE_Y_LOCATION'
       #     ][0] 
        #    / 10 ** magnitude
        #) * (10 ** magnitude)

#Random perturbation
#raw_data = cy.node.get_properties(network='CURRENT', nodeList='all', propertyList='Y Location')
#for entry in raw_data:
 #   random = (randint(0,10000) - 5000) * 1.0E-12
  #  data[str(entry['SUID'])] = entry['visualProperties'][0]['value'] + random

#Doing both
raw_data = cy.node.get_properties(network='CURRENT', nodeList='all', propertyList='Y Location, Height')
heights = []
for item in raw_data:
    for prop in item['visualProperties']:
        if prop['visualProperty'] == 'NODE_HEIGHT':
            heights.append(prop['value'])
min_height = min(heights)
magnitude = floor(log10(min_height / 100))
                  
if magnitude <= 0:
    for entry in raw_data:
        random = (randint(0, 10000) - 5000) * (10 ** (magnitude - 5))
        data[str(entry['SUID'])] = round(
            [
                prop['value'] 
                for prop in entry['visualProperties'] 
                if prop['visualProperty'] == 'NODE_Y_LOCATION'
            ][0],
            -magnitude
        ) + random
else:
    for entry in raw_data:
        random = (randint(0, 10000) - 5000) * (10 ** (magnitude - 5))
        data[str(entry['SUID'])] = round(
            [
                prop['value'] 
                for prop in entry['visualProperties'] 
                if prop['visualProperty'] == 'NODE_Y_LOCATION'
            ][0] 
            / 10 ** magnitude
        ) * (10 ** magnitude) + random
        
data

{'464': 309.001699,
 '465': -325.10290200000003,
 '466': -322.695083,
 '467': 323.40169399999996,
 '468': -187.504845,
 '469': -552.8955129999999,
 '470': -310.59931600000004,
 '471': -156.19623099999998,
 '472': -562.503576,
 '473': -427.304889,
 '474': 69.195513,
 '475': 171.902759,
 '476': -45.599669,
 '477': -451.40245899999996,
 '478': -57.699912000000005,
 '479': -572.099918,
 '480': -574.60314,
 '481': -444.198834,
 '482': -427.297117,
 '483': -437.003324,
 '484': -463.501148,
 '485': -303.39618099999996,
 '486': -593.8034809999999,
 '487': -448.995646,
 '488': 152.700327,
 '489': -38.403715999999996,
 '490': -548.1039910000001,
 '491': 71.596172,
 '492': -165.79740600000002,
 '493': -305.795101}

In [20]:
#Cluster data
clusters = [{'members': [item[0]], 'values': [item[1]]} for item in data.items()]
iterations = 0

print('Interation ' + str(iterations) + ':')
variance_within = 0
for cluster in clusters:
    print(cluster['members'])
    if len(cluster['values']) > 1:
        variance_within += variance(cluster['values'])

print('Variance within: ' + str(variance_within))
if len(clusters) > 1:
    variance_between = variance([mean(cluster['values']) for cluster in clusters])
    print('Variance between: ' + str(variance_between))
    #ratio = variance_within / variance_between
    #print('Ratio: ' + str(ratio))
print()

while len(clusters) > 1:
    iterations += 1
    #Find closest clusters
    smallest_diff = float('Infinity')
    i_coord = None
    j_coord = None
    for i in range(len(clusters)-1):
        for j in range(i+1, len(clusters)):
            diff = abs(mean(clusters[i]['values']) - mean(clusters[j]['values']))
            if diff < smallest_diff:
                smallest_diff = diff
                i_coord = i
                j_coord = j
    #Merge closest clusters
    clusters[i_coord]['members'] += clusters[j_coord]['members']
    clusters[i_coord]['values'] += clusters[j_coord]['values']
    del(clusters[j_coord])
    
    #Print
    #print('Interation ' + str(iterations) + ':')
    #variance_within = 0
    for cluster in clusters:
     #   print(cluster['members'])
        if len(cluster['values']) > 1:
            variance_within += variance(cluster['values'])
    print(variance_within)   
    #print('Variance within: ' + str(variance_within))
    #if len(clusters) > 1:
     #   variance_between = variance([mean(cluster['values']) for cluster in clusters])
      #  print('Variance between: ' + str(variance_between))
      #  print(variance_between)
       # ratio = variance_within / variance_between
        #print('Ratio: ' + str(ratio))
    #print()
                
    

Interation 0:
['464']
['465']
['466']
['467']
['468']
['469']
['470']
['471']
['472']
['473']
['474']
['475']
['476']
['477']
['478']
['479']
['480']
['481']
['482']
['483']
['484']
['485']
['486']
['487']
['488']
['489']
['490']
['491']
['492']
['493']
Variance within: 0
Variance between: 76448.54622301349

3.020199199991113e-05
2.8774689871840775
8.636489589516632
17.29188460033355
28.846075779531013
43.5333271493706
69.69992005745158
106.41919977720451
153.7144797072327
226.9006294263654
346.17805983581104
496.83381940660774
685.1490769788202
977.1442625510449
1323.1955166419684
1738.4460343284
2234.604824918409
2915.1303128657305
3677.654731417838
4517.449055247157
5568.422895039554
6760.682692272568
10643.662898187846
18824.336483053128
30930.68792397077
49084.824701635414
76655.82289053002
118101.85265833845
194550.39888135192
